#### 0. Create 3 kernals, 2 images with 3 channels


In [1]:
import random as rd
import numpy as np
import struct
import math

KERNAL_SIZE = 3
IMG_SIZE    = 4

rd.seed(1234)



kernal0  = np.array([[rd.randint(0,2) for i in range(KERNAL_SIZE)] for j in range(KERNAL_SIZE)], dtype="float32")
kernal1  = np.array([[rd.randint(0,1) for i in range(KERNAL_SIZE)] for j in range(KERNAL_SIZE)], dtype="float32")
kernal2  = np.array([[rd.randint(0,1) for i in range(KERNAL_SIZE)] for j in range(KERNAL_SIZE)], dtype="float32")

img0_0    = np.array([[rd.randint(0,2) for i in range(IMG_SIZE)] for j in range(IMG_SIZE)], dtype="float32")
img0_1    = np.array([[rd.randint(0,2) for i in range(IMG_SIZE)] for j in range(IMG_SIZE)], dtype="float32")
img0_2    = np.array([[rd.randint(0,2) for i in range(IMG_SIZE)] for j in range(IMG_SIZE)], dtype="float32")

img1_0    = np.array([[rd.randint(0,2) for i in range(IMG_SIZE)] for j in range(IMG_SIZE)], dtype="float32")
img1_1    = np.array([[rd.randint(0,2) for i in range(IMG_SIZE)] for j in range(IMG_SIZE)], dtype="float32")
img1_2    = np.array([[rd.randint(0,2) for i in range(IMG_SIZE)] for j in range(IMG_SIZE)], dtype="float32")

weight    = np.array([[rd.randint(0,2) for i in range(2)] for j in range(2)], dtype="float32")


In [2]:
kernal0,kernal1,kernal2

(array([[1., 0., 0.],
        [0., 2., 0.],
        [2., 2., 0.]], dtype=float32),
 array([[0., 1., 0.],
        [0., 0., 0.],
        [1., 1., 1.]], dtype=float32),
 array([[0., 0., 0.],
        [0., 0., 1.],
        [0., 0., 1.]], dtype=float32))

In [3]:
img0_0,img1_0

(array([[0., 2., 2., 0.],
        [2., 2., 0., 1.],
        [2., 0., 2., 1.],
        [1., 1., 1., 0.]], dtype=float32),
 array([[1., 0., 1., 1.],
        [2., 0., 0., 1.],
        [0., 1., 0., 1.],
        [0., 2., 1., 1.]], dtype=float32))

In [4]:
weight

array([[0., 2.],
       [1., 0.]], dtype=float32)

#### 1. Do zero-padding and convolution related to signals


#### 0.Zero-padding, since reading into an array with raster scan order, place it into the correct position

In [5]:
zero_padded_img = np.zeros((IMG_SIZE+2,IMG_SIZE+2),dtype= 'float32')

In [6]:
zero_padded_img

array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.]], dtype=float32)

In [7]:

for i in range(IMG_SIZE):
    for j in range(IMG_SIZE):
        zero_padded_img[i+1,j+1] = img0_0[i,j]


In [8]:
zero_padded_img

array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 2., 2., 0., 0.],
       [0., 2., 2., 0., 1., 0.],
       [0., 2., 0., 2., 1., 0.],
       [0., 1., 1., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0.]], dtype=float32)

## Replication

In [9]:
replicated_img = np.zeros((IMG_SIZE+2,IMG_SIZE+2),dtype= 'float32')
for i in range(IMG_SIZE):
    for j in range(IMG_SIZE):
        pixel = img0_0[i,j]
        # Check 4 corners
        # 4 corners
        if i == 0 and j == 0:
            replicated_img[0][0] = pixel
            replicated_img[0][1] = pixel
            replicated_img[1][0] = pixel
            replicated_img[1][1] = pixel
        elif i == 0 and j == IMG_SIZE-1:
            replicated_img[1][4] = pixel
            replicated_img[1][5] = pixel
            replicated_img[0][4] = pixel
            replicated_img[0][5] = pixel
        elif i == IMG_SIZE -1 and j == 0:
            replicated_img[4][1] = pixel
            replicated_img[4][0] = pixel
            replicated_img[5][0] = pixel
            replicated_img[5][1] = pixel
        elif i == IMG_SIZE -1 and j == IMG_SIZE-1:
            replicated_img[4][4] = pixel
            replicated_img[4][5] = pixel
            replicated_img[5][4] = pixel
            replicated_img[5][5] = pixel
        elif i == 0:
            replicated_img[0][j+1] = pixel
            replicated_img[1][j+1] = pixel
        elif j == 0:
            replicated_img[i+1][0] = pixel
            replicated_img[i+1][1] = pixel
        elif i == IMG_SIZE -1:
            replicated_img[4][j+1] = pixel
            replicated_img[5][j+1] = pixel
        elif j == IMG_SIZE -1:
            replicated_img[i+1][4] = pixel
            replicated_img[i+1][5] = pixel
        else:
            replicated_img[i+1][j+1] = pixel


In [10]:
replicated_img

array([[0., 0., 2., 2., 0., 0.],
       [0., 0., 2., 2., 0., 0.],
       [2., 2., 2., 0., 1., 1.],
       [2., 2., 0., 2., 1., 1.],
       [1., 1., 1., 1., 0., 0.],
       [1., 1., 1., 1., 0., 0.]], dtype=float32)

In [11]:
def zero_padded_img(img):
    zero_padded_img = np.zeros((IMG_SIZE+2,IMG_SIZE+2),dtype= 'float32')
    for i in range(IMG_SIZE):
        for j in range(IMG_SIZE):
            zero_padded_img[i+1,j+1] = img[i,j]

    return zero_padded_img

def replication(img):
    replicated_img = np.zeros((IMG_SIZE+2,IMG_SIZE+2),dtype= 'float32')
    for i in range(IMG_SIZE):
        for j in range(IMG_SIZE):
            pixel = img[i,j]
            # Check 4 corners
            # 4 corners
            if i == 0 and j == 0:
                replicated_img[0][0] = pixel
                replicated_img[0][1] = pixel
                replicated_img[1][0] = pixel
                replicated_img[1][1] = pixel
            elif i == 0 and j == IMG_SIZE-1:
                replicated_img[1][4] = pixel
                replicated_img[1][5] = pixel
                replicated_img[0][4] = pixel
                replicated_img[0][5] = pixel
            elif i == IMG_SIZE -1 and j == 0:
                replicated_img[4][1] = pixel
                replicated_img[4][0] = pixel
                replicated_img[5][0] = pixel
                replicated_img[5][1] = pixel
            elif i == IMG_SIZE -1 and j == IMG_SIZE-1:
                replicated_img[4][4] = pixel
                replicated_img[4][5] = pixel
                replicated_img[5][4] = pixel
                replicated_img[5][5] = pixel
            elif i == 0:
                replicated_img[0][j+1] = pixel
                replicated_img[1][j+1] = pixel
            elif j == 0:
                replicated_img[i+1][0] = pixel
                replicated_img[i+1][1] = pixel
            elif i == IMG_SIZE -1:
                replicated_img[4][j+1] = pixel
                replicated_img[5][j+1] = pixel
            elif j == IMG_SIZE -1:
                replicated_img[i+1][4] = pixel
                replicated_img[i+1][5] = pixel
            else:
                replicated_img[i+1][j+1] = pixel

    return replicated_img


### Use Zeropadded for further testing

In [12]:
img0_0 = zero_padded_img(img0_0)
img0_1 = zero_padded_img(img0_1)
img0_2 = zero_padded_img(img0_2)

In [13]:
img0_0

array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 2., 2., 0., 0.],
       [0., 2., 2., 0., 1., 0.],
       [0., 2., 0., 2., 1., 0.],
       [0., 1., 1., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0.]], dtype=float32)

#### 2. Sum add convolution, fill up 4 max pooling results.


In [14]:
convoluted_results = np.zeros((IMG_SIZE,IMG_SIZE),dtype= 'float32')

In [15]:
for i in range(IMG_SIZE):
    for j in range(IMG_SIZE):
        conv1,conv2,conv3 = np.float32(0),np.float32(0),np.float32(0)
        for k in range(KERNAL_SIZE):
            for l in range(KERNAL_SIZE):
                conv1 += kernal0[k,l] * img0_0[i+k,j+l]
                conv2 += kernal1[k,l] * img0_1[i+k,j+l]
                conv3 += kernal2[k,l] * img0_2[i+k,j+l]

        convoluted_results[i,j] = conv1 + conv2 + conv3


In [16]:
def convolution(img0_0, img0_1,img0_2):
    convoluted_results = np.zeros((IMG_SIZE,IMG_SIZE),dtype= 'float32')
    for i in range(IMG_SIZE):
        for j in range(IMG_SIZE):
            conv1,conv2,conv3 = np.float32(0),np.float32(0),np.float32(0)
            for k in range(KERNAL_SIZE):
                for l in range(KERNAL_SIZE):
                    conv1 += kernal0[k,l] * img0_0[i+k,j+l]
                    conv2 += kernal1[k,l] * img0_1[i+k,j+l]
                    conv3 += kernal2[k,l] * img0_2[i+k,j+l]

            convoluted_results[i,j] = conv1 + conv2 + conv3

    return convoluted_results

In [17]:
type(conv1)

numpy.float32

In [18]:
convoluted_results

array([[ 6., 20., 11.,  4.],
       [11., 12., 10., 14.],
       [11., 13., 16.,  5.],
       [ 2.,  7.,  3.,  4.]], dtype=float32)

### Equalization(Zeropadding and replication)

In [19]:
opt = 1
equalization_img = np.zeros((IMG_SIZE,IMG_SIZE),dtype= 'float32')
for xptr in range(0,4):
    for yptr in range(0,4):
        sum = 0
        for k_xptr in range(0,3):
            for k_yptr in range(0,3):
                y_offset = yptr + k_yptr
                x_offset = xptr + k_xptr

                if opt == 0 or opt == 2: # Replication
                    # Corners
                    if y_offset == 0 and x_offset == 0:
                        sum += convoluted_results[0,0]
                    elif y_offset == 5 and x_offset == 0:
                        sum += convoluted_results[0,3]
                    elif y_offset == 0 and x_offset == 5:
                        sum += convoluted_results[3,0]
                    elif y_offset == 5 and x_offset == 5:
                        sum += convoluted_results[3,3]
                    # Boundaries
                    elif x_offset == 0:
                        sum += convoluted_results[0,y_offset-1]
                    elif x_offset == 5:
                        sum += convoluted_results[3,y_offset-1]
                    elif y_offset == 0:
                        sum += convoluted_results[x_offset-1,0]
                    elif y_offset == 5:
                        sum += convoluted_results[x_offset-1,3]
                    else:
                        sum += convoluted_results[x_offset-1,y_offset-1]
                elif opt == 1 or opt == 3: # Zeropadding
                    if y_offset == 0 or x_offset == 0 or y_offset == 5 or x_offset == 5:
                        sum += 0
                    else:
                        sum += convoluted_results[x_offset-1,y_offset-1]

        equalization_img[xptr,yptr] = sum/9

In [20]:
convoluted_results

array([[ 6., 20., 11.,  4.],
       [11., 12., 10., 14.],
       [11., 13., 16.,  5.],
       [ 2.,  7.,  3.,  4.]], dtype=float32)

In [21]:
equalization_img

array([[ 5.4444447,  7.7777777,  7.888889 ,  4.3333335],
       [ 8.111111 , 12.222222 , 11.666667 ,  6.6666665],
       [ 6.2222223,  9.444445 ,  9.333333 ,  5.7777777],
       [ 3.6666667,  5.7777777,  5.3333335,  3.1111112]], dtype=float32)

### Max pooling

In [22]:
max_pooled_result = np.zeros((2,2),dtype='float32')
for i in range(0,4,2):
    for j in range(0,4,2):
        temp_max = np.float32(0)
        for k in range(0,2):
            for l in range(0,2):
                if temp_max < convoluted_results[i+k,j+l]:
                    temp_max = convoluted_results[i+k,j+l]

        max_pooled_result[i//2,j//2] = temp_max


In [23]:
def max_pooling(convoluted_results):
    max_pooled_result = np.zeros((2,2),dtype='float32')
    # Finish this with 2 cycles
    for i in range(0,4,2):
        # Parallelize this using 2 comparators
        for j in range(0,4,2):
            temp_max = np.float32(0)
            for k in range(0,2):
                for l in range(0,2):
                    if temp_max < convoluted_results[i+k,j+l]:
                        temp_max = convoluted_results[i+k,j+l]

            max_pooled_result[i//2,j//2] = temp_max

    return max_pooled_result


In [24]:
type(temp_max)

numpy.float32

In [25]:
max_pooled_result

array([[20., 14.],
       [13., 16.]], dtype=float32)

In [26]:
weight

array([[0., 2.],
       [1., 0.]], dtype=float32)

#### 3. Fully connected layers with matrix multiplication
### Finding the min-max of fully as computing the fully connected layers.

In [27]:
fully_connected_result = np.zeros((4,1),dtype='float32')

LARGE_NUMBER = 99999999999999999999999999999
SMALL_NUMBER = -99999999999999999999999999999

min = np.float32(LARGE_NUMBER)
max = np.float32(SMALL_NUMBER)

for i in range(2):
    for j in range(2):
        partial_mult = 0
        for k in range(2):
            partial_mult += max_pooled_result[i,k] * weight[k,j]

        if partial_mult > max:
            max = partial_mult
        if partial_mult < min:
            min = partial_mult

        fully_connected_result[i*2+j] = partial_mult


In [28]:
def fully_connection(max_pooled_result,weight):
    fully_connected_result = np.zeros((4,1),dtype='float32')
    min = np.float32(LARGE_NUMBER)
    max = np.float32(SMALL_NUMBER)

    for i in range(2):
        for j in range(2):
            partial_mult = 0
            for k in range(2):
                partial_mult += max_pooled_result[i,k] * weight[k,j]

            if partial_mult > max:
                max = partial_mult
            if partial_mult < min:
                min = partial_mult

            fully_connected_result[i*2+j] = partial_mult

    return fully_connected_result


In [29]:
type(min)

numpy.float64

In [30]:
fully_connected_result = fully_connection(max_pooled_result,weight)

In [31]:
fully_connected_result

array([[14.],
       [40.],
       [16.],
       [26.]], dtype=float32)

In [32]:
type(max),min

(numpy.float64, 14.0)

#### 5. Do the min-max Normalization


In [33]:
x_scaled = np.zeros((4,1),dtype='float32')
for i,x in enumerate(fully_connected_result):
    x_scaled[i] = (x-min) / (max-min)

In [34]:
def min_max_norm(fully_connected_result):
    x_scaled = np.zeros((4,1),dtype='float32')
    for i,x in enumerate(fully_connected_result):
        x_scaled[i] = (x-min) / (max-min)

    return x_scaled

In [35]:
x_scaled = min_max_norm(fully_connected_result)

In [36]:
x_scaled

array([[0.        ],
       [1.        ],
       [0.07692308],
       [0.46153846]], dtype=float32)

#### 6. Do the sigmoid activation function

## Sigmoid

In [37]:
activated_value = np.zeros((4,1),dtype='float32')

In [38]:
for i,e in enumerate(x_scaled):
    activated_value[i] = 1/(1+np.exp(-e,dtype='float32'))

In [39]:
def sigmoid(x_scaled):
    activated_value = np.zeros((4,1),dtype='float32')
    for i,e in enumerate(x_scaled):
        activated_value[i] = 1/(1+np.exp(-e,dtype='float32'))

    return activated_value

In [40]:
activated_value

array([[0.5       ],
       [0.7310586 ],
       [0.5192213 ],
       [0.61337906]], dtype=float32)

## Tanh

In [41]:
for i,e in enumerate(x_scaled):
    activated_value[i] = (np.exp(e,dtype='float32') - np.exp(-e,dtype='float32')) \
    / (np.exp(e,dtype='float32') + np.exp(-e,dtype='float32'))

In [42]:
def tanh(x_scaled):
    activated_value = np.zeros((4,1),dtype='float32')
    for i,e in enumerate(x_scaled):
        activated_value[i] = 1/(1+np.exp(-e,dtype='float32'))

    return activated_value

## Hex representaion conversion

In [43]:
activated_value

array([[0.        ],
       [0.7615942 ],
       [0.07677177],
       [0.43133733]], dtype=float32)

In [44]:
activated_value[0][0]

0.0

In [45]:
value = activated_value[0][0]
# Get the hexadecimal representation (as a string)
float_as_int = np.float32(value).view(np.int32)
hex_representation = format(value.view(np.int32), 'x')

In [46]:
hex_representation

'0'